In [ ]:
# hide
!nvidia-smi

Wed Jun  2 09:59:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# hide
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai transformers datasets wandb rouge-score nltk
    !pip install -q git+git://github.com/aikindergarten/fasthugs.git@sum

     |████████████████████████████████| 204kB 7.7MB/s 
     |████████████████████████████████| 2.3MB 13.8MB/s 
     |████████████████████████████████| 235kB 37.4MB/s 
     |████████████████████████████████| 1.8MB 36.7MB/s 
     |████████████████████████████████| 1.5MB 37.3MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 3.3MB 31.5MB/s 
     |████████████████████████████████| 901kB 34.1MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
     |████████████████████████████████| 245kB 35.1MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 174kB 39.8MB/s 
     |████████████████████████████████| 133kB 39.6MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 


In [ ]:
#all_slow

# Summarization

In [ ]:
#hide
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, MarianConfig, AutoTokenizer, AutoConfig
from datasets import load_dataset, load_metric, concatenate_datasets
from fastai.text.all import *
from fastai.callback.wandb import *
import nltk
from fasthugs.learner import TransLearner, GeneratePreds
from fasthugs.data import TransformersTextBlock, TextGetter, KeyGetter, get_splits
from fasthugs.metrics import RougeScore

## Setup

In [ ]:
model_name = "t5-small"

max_len = 512
bs = 16
val_bs = bs*2

lr = 2e-5

In [ ]:
#hide_output
# datasets = load_dataset("xsum")
train_ds = load_dataset("xsum", split='validation')

Using custom data configuration default


Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499. Subsequent calls will reuse this data.


## Dataloaders

In [ ]:
#hide_output
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# splits = get_splits(datasets)
# train_ds = concatenate_datasets([datasets['train'], datasets['validation']])

In [ ]:
splits = RandomSplitter()(train_ds)

In [ ]:
@ItemTransform
def untuple1(x):
    return (*x[0], )

In [ ]:
dblock = DataBlock(
    blocks = [TransformersTextBlock(tokenizer=tokenizer, do_targets=True, with_labels=True)],
    get_x=TextGetter('document', 'summary', prefix1='summarize: '),
    item_tfms=untuple1,
    splitter=IndexSplitter(splits[1]))

In [ ]:
#hide
# dblock.summary(train_ds)

In [ ]:
%%time
dls = dblock.dataloaders(train_ds, bs=bs, val_bs=bs*2, shuffle=True)

CPU times: user 6.79 s, sys: 1.34 s, total: 8.13 s
Wall time: 15.6 s


In [ ]:
#hide
b = dls.one_batch()
b

({'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
  'input_ids': tensor([[21603,    10, 11768,  ..., 23080,     7,     1],
          [21603,    10,   621,  ...,     3,    60,     1],
          [21603,    10,   216,  ...,   826,     8,     1],
          ...,
          [21603,    10,   901,  ...,    12,   253,     1],
          [21603,    10, 10582,  ...,    34,    91,     1],
          [21603,    10,    37,  ...,    27,    31,     1]], device='cuda:0'),
  'labels': tensor([[18539,  3763, 24570, 15497,     6,  2783,  2213,   262,  2341,    23,
            5440,    15,    11,  8929, 16023,  2291,   446,   439, 11768,   697,
              33,   859,     8,   168,    18,  5661,  8519,    16,  8288,    12,
              43,  1204,  6120,    16,     8,  5286,    31,     7, 13753

In [ ]:
#collapse
dls.show_batch(max_n=4)

,text,text_
0,"summarize: Rowling, who was made an OBE in 2001, has now become a member of the elite Order of the Companions of Honour. Judy Murray, mother of world tennis number one Sir Andy, was made an OBE. And Brit Award winner Sande, who was raised in Alford, Aberdeenshire, was made an MBE. Scottish comedian Billy Connolly was given a knighthood. Judy Murray adds to the honours already handed to her two sons. The tennis coach and former captain of Great Britain's Fed Cup team is being recognised for her work to grow the sport and for encouraging more women into sport. Sir Andy Murray, was knighted in the New Year Honours List, while his brother Jamie was made an OBE last year. Harry Potter author Rowling, who was made an OBE in 2001, becomes a member of the Order of the Companions of Honour, which has a maximum of 65","Tennis coach Judy Murray, pop star Emeli Sande and Harry Potter author JK Rowling are among the well-known faces in Scotland to have received awards in the Queen's Birthday Honours."
1,"summarize: Rooney, 31, played 559 times for United, scoring 253 goals. He won five Premier League titles and each of the Champions League, Europa League and FA Cup once after joining from Everton for £27m in 2004. Rooney, who has signed a two-year deal, said he was ""ecstatic"" and his ""first game back will be an emotional day"". ""It's a great feeling to be back. I cannot wait to meet the lads, get on the training pitch and then get on the pitch to play,"" he added. Rooney's return comes as United look set to sign Everton striker Romelu Lukaku, with a £75m deal for the Belgium international agreed between the two clubs. Everton confirmed Rooney will wear the number 10 shirt previously worn by Lukaku. ""I'm not just coming back because it's the team I support, the team I grew up playing for - I'm coming back because I","Manchester United record goalscorer Wayne Rooney has rejoined Everton for an undisclosed fee, 13 years after leaving the Merseyside club."
2,"summarize: The last week of Yvette Cooper's leadership campaign isn't exactly how she predicted it. She has spent more time in Parliament than on the stump. As shadow home secretary she has been leading her party's response to the refugee crisis - initially calling on the government to do more, and now pressing for some of those families who have already fled to Europe to be resettled here. This has helped, rather than harmed, her leadership chances but has also meant more conventional campaigning has had to be discarded. ""I tend to respond most strongly to the most serious issues but I'm trying to keep this apart from the leadership contest, and trying to get cross party agreement to an appropriate response to the refugee crisis,"" she tells me. While I met her rivals ""on the road"" at rallies and meetings, we meet in her parliamentary office. She was on","In the fourth of a series of in-depth profiles of the Labour leadership candidates, Iain Watson catches up with Yvette Cooper."
3,"summarize: Media playback is not supported on this device The home team took the lead when right-back Ben Marshall tucked in a low shot. Victor Moses's equaliser squirmed in and Payet's free-kick put West Ham ahead before Blackburn were reduced to 10 men by Chris Taylor's red card. Emmanuel Emenike converted twice either side of Cheikhou Kouyate's dismissal, then Payet added a superb solo goal. The Hammers, who last won the FA Cup in 1980, will travel to either Shrewsbury or Manchester United in the last eight. It is only the second time in the past 10 seasons that they have reached the quarter-finals. Read how the action unfolded at Ewood Park West Ham have not won a major trophy for 36 years but, in their final season at the Boleyn Ground, are hoping that they can take the FA Cup with them to the Olympic Stadium. Hammers manager Slaven",Dimitri Payet produced a virtuoso performance as West Ham fought back to ease into the FA Cup quarter-finals at Championship side Blackburn.


In [ ]:
#hide
# import wandb

# WANDB_NAME = f'{ds_name}-{model_name}'
# GROUP = f'{ds_name}-{model_name}-simple-{lr:.0e}'
# NOTES = f'finetuning {model_name} with RAdam lr={lr:.0e}'
# CONFIG = {}
# TAGS =[model_name, ds_name, 'radam']

In [ ]:
#hide
# wandb.init(reinit=True, project="fasthugs", entity="fastai_community",
#            name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

## Training

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
learn = TransLearner(dls, model, loss_func=noop)

In [ ]:
learn.add_cb(RougeScore(tokenizer));

In [ ]:
#hide
learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, RougeScore, Recorder, ProgressCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : [TransCallback]
         - after_pred     : [TransCallback, RougeScore]
         - after_loss     : [TransCallback]
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]
     - after_valid

In [ ]:
learn.validate()

(#5) [3.887169599533081,0.2408951647750222,0.056876429032900334,0.21799074098609333,0.2180012913860141]

In [ ]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,rougeLsum,time
0,2.848689,2.597462,0.297854,0.100571,0.271334,0.278742,03:19
1,2.711888,2.562764,0.302310,0.103245,0.275056,0.282703,03:19


In [ ]:
#hide
def display_validation_results(res):
    m_names = ['train_loss', 'valid_loss']+[m.name for m in learn.metrics]
    res_df = pd.DataFrame.from_dict({k:[v] for k,v in zip(m_names, res)})
    display_df(res_df)

In [ ]:
%%time
res = learn.validate()
display_validation_results(res)

,train_loss,valid_loss,rouge1,rouge2,rougeL
0,2.562764,0.302342,0.103331,0.274998,0.282674


CPU times: user 17.6 s, sys: 391 ms, total: 18 s
Wall time: 19.8 s


So far we computed predictions using single forward pass so token generated at timestep $t$ has access to reference tokens $0:t-1$. But this at inference time we will generate autoregressively previously generated tokens are used to generate the next one. Let's evaluete the model with this more realistic procedure. This can be done by adding `GeneratePreds` callback:

In [ ]:
%%time
res = learn.validate(cbs=GeneratePreds())
display_validation_results(res)

,train_loss,valid_loss,rouge1,rouge2,rougeL
0,2.562764,0.26428,0.067737,0.209218,0.209238


CPU times: user 40.5 s, sys: 515 ms, total: 41 s
Wall time: 42.9 s


In [ ]:
sample = train_ds[0]
document_text = ' '.join(sample['document'].split('\n'))
print(f"Document:\n{document_text}")
print(f"\nReference summary: {sample['summary']}")
inp = tokenizer('summarize: '+sample['document'], return_tensors='pt')
pred = learn.generate(inp['input_ids'].to(dls.device))
out = tokenizer.decode(pred[0].cpu(), skip_special_tokens=True)
print(f"\nPredicted summary: {out}")

Document:
The country's consumer watchdog has taken Apple to court for false advertising because the tablet computer does not work on Australia's 4G network. Apple's lawyers said they were willing to publish a clarification. However the company does not accept that it misled customers. The Australian Competition and Consumer Commission (ACCC) said on Tuesday: "Apple's recent promotion of the new 'iPad with wi-fi + 4G' is misleading because it represents to Australian consumers that the product can, with a sim card, connect to a 4G mobile data network in Australia, when this is not the case." The watchdog then lodged a complaint at the Federal Court in Melbourne. At a preliminary hearing, Apple lawyer Paul Anastassiou said Apple had never claimed the device would work fully on the current 4G network operated by Telstra. Apple says the new iPad works on what is globally accepted to be a 4G network. The matter will go to a full trial on 2 May. The Apple iPad's third version went on sale e